# Word Analogies

Word embeddings allow us to process text data in all kinds of interesting ways. 
One experiment is to use code to solve _word analogies_.

> Solving a word analogy "A is to B as X is to Y" means to find one of the parameters, given the other three.

For example:
- London is to UK as Moscow is to what?
- Cat is to kitten as dog is to what?

> Word analogies can be solved using word embeddings

What is the point of this?
- There seems to be little practical application
- But it can help 
    - To understand what word vectors represent
    - To determine if you've found a useful set of word embeddings

Mathematically, that means finding the vector between $a$ and $b$, then adding that to $x$.

# TODO diagram of adding analogy vector to source

Firstly, let's get some pre-trained word embeddings from an extremely widely used embedding model named BERT:

In [ ]:
from transformers import BertModel, BertTokenizer

# %% GET BERT
model_name = 'bert-base-uncased' 
model = BertModel.from_pretrained(model_name) # TODO get BERT model from huggingface
bert_tokenizer = BertTokenizer.from_pretrained(model_name) # TODO get BERT tokeniser from huggingface

# EXAMPLE TOKENISATION
sentence = "Now I want to know what does this vector refers to in dictionary"
tokens = bert_tokenizer.encode(sentence) # TODO encode the sentence

In [ ]:
print(model.modules)

In that list of modules, you can see that the first one is the embedding layer. 
The weights of this layer are the input representation that BERT has learnt for each word.
These are the pre-trained embeddings that we will use.

In [ ]:

embedding_matrix = model.embeddings.word_embeddings.weight # TODO get weight parameters from model
embedding_matrix = embedding_matrix.detach() # TODO detach parameters from graph

n_embeddings = 30000
embedding_matrix = embedding_matrix[:n_embeddings] # TODO get the first n_embeddings

print("Embedding shape:", embedding_matrix.shape) # TODO print embedding matrix shape


Now we have the embeddings, we want to determine which row corresponds to which token. We can get this mapping from the pre-trained BERT tokeniser:

In [ ]:
embedding_labels = list(bert_tokenizer.ids_to_tokens.values())[:n_embeddings] # TODO get the names of the tokens from the tokeniser

Let's quickly define a helper function to visualise our embeddings using Tensorboard:

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from time import time

def visualise_embeddings(embeddings, labels=None, label_names="Label"):
    print("Embedding")

    writer = SummaryWriter() # TODO initialise tensorboard summarywriter
    start = time()
    writer.add_embedding( # TODO add embeddings to tensorboard
        mat=embeddings,
        metadata=labels,
        metadata_header=label_names
    )
    print(f"Total time:", time() - start)

    print("Embedding done")

visualise_embeddings(embedding_matrix, embedding_labels) # TODO call visualise_embeddings

To determine the vector that represents the transformation between $a$ & $b$, we'll need to firstly get the embedding for each of them:

In [ ]:
def get_word_embedding(word):

    tokens_to_ids = {token: id for id, token in bert_tokenizer.ids_to_tokens.items()} # TODO create a mapping from the tokeniser's ids_to_tokens attribute by reversing it with a dictionary comprehension

    token_id = tokens_to_ids[word] # TODO get the id from the tokeniser
    embedding = embedding_matrix[token_id] # TODO index embedding for this id out of the embedding matrix
    return embedding

example_word_embedding = get_word_embedding("apple")
print(example_word_embedding)

To find the closest vector to an embedding, we'll need to compare its distace to all other token embeddings. An effective way to do that is by taking their cosine similarity. 

## TODO diagram of comparing word vectors with cosine similarity

We could implement the cosine similarity ourselves, but we can also get a function to do that off the shelf, from the `torchmetrics` library. You can check out the documentation [here](https://torchmetrics.readthedocs.io/en/stable/pairwise/cosine_similarity.html).

In [ ]:
!pip install torchmetrics

Often, the nearest token to the solved analogy embedding is the token that you started with or its plural.
So, we might want to get more than just the closest one. 

Now let's define a function to get the nearest $n$ tokens to an embedding:

In [ ]:
import torch
import torchmetrics

def get_nearest_n_tokens_from_embedding(embedding, n=20):
    # cosine similarity from d_embedding to embedding of all words
    similarity = torchmetrics.functional.pairwise_cosine_similarity( # TODO take the pairwise cosine distance
        embedding.unsqueeze(0), embedding_matrix).squeeze()
    similarity_idx = reversed(torch.argsort(similarity, dim=0)) # TODO argsort by similarity score
    print(similarity_idx.shape)
    similarity_idx = similarity_idx[:n] # TODO slice out the indexes of the top n
    return [list(bert_tokenizer.ids_to_tokens.values())[idx] for idx in similarity_idx] # TODO get the top n most similar tokens from the tokeniser

get_nearest_n_tokens_from_embedding(example_word_embedding)


Now, let's implement a function to solve the analogy:

In [ ]:
def analogy_solver(a, b, c, embedding_matrix, labels, n=5):
    """
    Solves A is to B what C is to D, given, A, B & C, returning D

    """

    # GET EMBEDDINGS FOR KNOWN WORDS
    a_embedding = get_word_embedding(a)
    b_embedding = get_word_embedding(b)

    # GET TRANSFORMATION APPLIED
    transformation_vector = b_embedding - a_embedding # TODO calculate vector difference between a and b

    c_embedding = get_word_embedding(c) # TODO get word embedding of c
    print(c_embedding.shape)
    d_embedding = c_embedding + transformation_vector # TODO add difference between a and b to c
    print(d_embedding.shape)
    nearest_tokens = get_nearest_n_tokens_from_embedding(d_embedding, n=n+1) # TODO get n+1 nearest tokens (n+1 because the most similar to c is often itself)
    for d in nearest_tokens: # TODO for each nearest token
        if d == c: # TODO skip if d == c
            continue
        print(f"{a} is to {b} as {c} is to {d}")
    print()

Now let's use that to solve a few analogies:

In [ ]:
analogy_solver("man", "woman", "king", embedding_matrix, embedding_labels)
analogy_solver("london", "uk", "moscow", embedding_matrix, embedding_labels)
analogy_solver("puppy", "dog", "kitten", embedding_matrix, embedding_labels)

You can see that the analogies seem to work (roughly)